In [1]:
!pip install crewai
!pip install requests
!pip install langchain_google_genai



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 677.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 359.2 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 6.7 MB/s eta 0:0

In [2]:
import crewai

In [3]:
!pip install crewai-tools # Install the correct package
from crewai_tools import (SerperDevTool, WebsiteSearchTool) # Import the classes

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 11.3 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=1469764ff0d5df4903f9d2468c191d237aa20828e52f33176917c47a2622f5b9
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [5]:
import os

In [7]:
from google.colab import userdata
# Retrieve your Google API key
google_api_key = userdata.get('Gemini_API')
os.environ["GOOGLE_API_KEY"] = google_api_key
# Retrieve your Serper API key
serper_api_key = userdata.get('SERPER_API_KEY')
os.environ["SERPER_API_KEY"] = serper_api_key

In [8]:
## call the gemini models
gemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                           verbose=True,
                           temperature=0.5,
                           google_api_key=google_api_key)


In [9]:
# Initialize SerperDevTool
serper = SerperDevTool(api_key=serper_api_key)

In [ ]:
#from langchain.utilities import SerpAPIWrapper

In [ ]:
#!pip install google-search-results

In [10]:
from crewai import Agent, Task, Crew

In [17]:
def perform_search(query):
    if not query:
        return {"error": "Query parameter is missing"}

    try:
        search_results = serper.run()
        #search_results = serper.search(query)
        print("Search Results:", search_results)  # Debugging output

        # Check if the response is an error message or data
        if isinstance(search_results, dict) and 'message' in search_results:
            return {"error": search_results['message']}

        return search_results.get('results', search_results)  # Adjust based on actual structure
    except Exception as e:
        return {"error": str(e)}

# Function to process RAG
def rag_process(search_results, user_query):
    if isinstance(search_results, dict) and 'error' in search_results:
        return f"Error during search: {search_results['error']}"

    if not isinstance(search_results, list):
        return "Unexpected search results format"

    try:
        context = " ".join([result.get("snippet", "") for result in search_results])
        response = gemini.generate(
            prompt=f"Based on the following context, answer the dermatology query: {user_query}\nContext: {context}",
            max_tokens=150
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"Error during response generation: {str(e)}"




def handle_input(user_query):
    search_results = perform_search(user_query)
    response = rag_process(search_results, user_query)
    return response


In [12]:
def create_dermatology_agent():
    agent = Agent(
        role='Dermatologist',
        goal='Provide accurate and helpful dermatology-related information and advice.',
        backstory="""You are a dermatologist specializing in providing information and advice on skin conditions,
        treatments, and skincare practices. Your goal is to assist users with their dermatology-related queries
        by leveraging both search results and advanced language models.""",
        verbose=True,
        llm=gemini,
        tools=[serper],
        allow_delegation=True
    )
    return agent

In [13]:
# Define the task
def create_dermatology_task(agent):
    task = Task(
        description='Handle dermatology-related user queries with RAG and search',
        expected_output='A detailed and accurate response based on search results and RAG processing',
        agent=agent,
        #llm=gemini,
        tools=[serper]
        #allow_delegation=True
    )
    return task

In [14]:
from crewai import Crew, Process
# Create Crew instance and run task
def create_crew(agent, task):
    crew = Crew(
        agents=[agent],
        tasks=[task],
        verbose=2,
        process=Process.sequential
    )
    return crew


In [15]:
def get_user_input():
    #return input("Please enter your dermatology-related question: ")
    return input("Please ask Dermatology related question")



In [16]:
def main():
    # Create agent and task
    agent = create_dermatology_agent()
    task = create_dermatology_task(agent)

    # Create Crew instance
    crew = create_crew(agent, task)

    while True:
        user_query = get_user_input()
        if user_query.lower() in ['exit', 'quit']:
            print("Exiting...")
            break

        # Process the query with Crew
        result = handle_input(user_query)
        print(result)

# Run the main function
if __name__ == "__main__":
    main()

Please ask Dermatology related questionHow to deal with red spots on face
Using Tool: Search the internet
Search Results: {'message': 'Missing query parameter', 'statusCode': 400}
Error during search: Missing query parameter


KeyboardInterrupt: Interrupted by user